<a href="https://colab.research.google.com/github/abhay43/DL_Code/blob/master/Human%20Pose%20Detection%20using%20Mask%20RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
def unzipme(location):
  import zipfile
  with zipfile.ZipFile(location) as zip_ref:
    zip_ref.extractall()
    print('done')


In [4]:
unzipme('/content/drive/My Drive/Colab Notebooks/Dataset - Human Pose Estimation')

done


In [5]:
unzipme('/content/Pose_Dataset.zip')


done


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [7]:
train_data = pd.read_csv('/content/Pose_Dataset/train_joints_coords.csv', header=None)
test_data = pd.read_csv('/content/Pose_Dataset/test_joints_coords.csv', header = None)

In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4429 entries, 0 to 4428
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4429 non-null   object
 1   1       4429 non-null   int64 
 2   2       4429 non-null   int64 
 3   3       4429 non-null   int64 
 4   4       4429 non-null   int64 
 5   5       4429 non-null   int64 
 6   6       4429 non-null   int64 
 7   7       4429 non-null   int64 
 8   8       4429 non-null   int64 
 9   9       4429 non-null   int64 
 10  10      4429 non-null   int64 
 11  11      4429 non-null   int64 
 12  12      4429 non-null   int64 
 13  13      4429 non-null   int64 
 14  14      4429 non-null   int64 
dtypes: int64(14), object(1)
memory usage: 519.1+ KB


In [9]:
!pip uninstall keras

Uninstalling Keras-2.3.1:
  Would remove:
    /tensorflow-1.15.2/python3.6/Keras-2.3.1.dist-info/*
    /tensorflow-1.15.2/python3.6/docs/*
    /tensorflow-1.15.2/python3.6/keras/*
Proceed (y/n)? y
  Successfully uninstalled Keras-2.3.1


In [10]:
!pip install q keras==2.1.0

     |████████████████████████████████| 307kB 7.5MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.0 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [12]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 116.77 MiB | 18.77 MiB/s, done.
Resolving deltas: 100% (564/564), done.


In [13]:
cd Mask_RCNN

/content/Mask_RCNN


In [14]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/__init__.py -> build/bdist.linux-x86_64/egg/mr

In [15]:
!pip show mask-rcnn

Name: mask-rcnn
Version: 2.1
Summary: Mask R-CNN for object detection and instance segmentation
Home-page: https://github.com/matterport/Mask_RCNN
Author: Matterport
Author-email: waleed.abdulla@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages/mask_rcnn-2.1-py3.6.egg
Requires: 
Required-by: 


In [16]:
pwd

'/content/Mask_RCNN'

In [ ]:
class pose_detection:
  